In [1]:
import pandas as pd
import numpy as np

In [6]:
cali_layoffs = pd.read_csv('california_warn_raw.csv')
cali_population = pd.read_csv('county_population.csv', delimiter='\t', thousands=',')
#Audit Logging - can do it on slack

population data from: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html#par_textimage_242301767

In [3]:
cali_layoffs.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,Closure Permanent
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608.0,Closure Permanent
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104.0,Closure Permanent
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,Layoff Temporary
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91.0,Layoff Temporary


In [4]:
cali_layoffs.tail()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
9502,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",San Francisco,No County Name,86.0,Layoff Permanent
9503,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Santa Monica,No County Name,49.0,Closure Permanent
9504,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Venice,No County Name,11.0,Closure Permanent
9505,06/29/2015,08/28/2015,06/30/2015,"Safeway, Inc.",Pleasanton,No County Name,18.0,Layoff Unknown at this time
9506,06/30/2015,07/22/2015,06/30/2015,Medtronic Ablation Frontiers LLC,Carlsbad,No County Name,50.0,Closure Permanent


## California 2020 Layoffs

We are selecting 2020 records by isolating the year on the `effective date` column.
By choosing `effective date` we make sure that these layoffs actually occurred in 2020, as some layoff notices can be issues much earlier the layoff date.

In [7]:
cali_layoffs['Year'] = cali_layoffs['Effective Date'].str[-4:]
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.replace('at thistime', 'at this time')
cali_layoffs['County'] = cali_layoffs['County'].str.strip()
cali_layoffs.loc[~(cali_layoffs['County'].str.endswith('County')),'County'] = cali_layoffs['County'] + " County"
cali_2020 = cali_layoffs[cali_layoffs['Year'] == '2020']
cali_2020['Employees'] = cali_2020['Employees'].astype('Int64')
cali_2020.tail()

<ipython-input-7-c4817812f643>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2020['Employees'] = cali_2020['Employees'].astype('Int64')


,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
5725,10/30/2019,01/03/2020,11/01/2019,"Ashford University, LLC",San Diego,San Diego County,218,Layoff Permanent,2020
5727,11/01/2019,01/10/2020,11/01/2019,American General Life Insurance Company,Pasadena,Los Angeles County,40,Closure Permanent,2020
5744,10/24/2019,01/01/2020,10/28/2019,"Hotel Del Coronado, Curio Collection By Hilton...",Coronado,San Diego County,160,Layoff Temporary,2020
5747,10/17/2019,05/24/2020,10/23/2019,Wells Fargo,Concord,Contra Costa County,357,Layoff Permanent,2020
5789,09/26/2019,01/16/2020,09/30/2019,"Forever 21 Logistics, LLC",Los Angeles,Los Angeles County,1170,Closure Permanent,2020


In [18]:
no_county = cali_layoffs[cali_layoffs['County'] == 'No County Name County']
no_county

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
7390,06/30/2016,08/31/2016,07/01/2016,DCS Facility Services,Antelope,No County Name County,55.0,Closure Permanent,2016
7391,06/30/2016,08/31/2016,07/01/2016,DCS Facility Services,Bakersfield,No County Name County,22.0,Closure Permanent,2016
7392,06/30/2016,08/31/2016,07/01/2016,DCS Facility Services,Brentwood,No County Name County,22.0,Closure Permanent,2016
7393,06/30/2016,08/31/2016,07/01/2016,DCS Facility Services,Chico,No County Name County,22.0,Closure Permanent,2016
7394,06/30/2016,08/31/2016,07/01/2016,DCS Facility Services,Clovis,No County Name County,33.0,Closure Permanent,2016
...,...,...,...,...,...,...,...,...,...
9502,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",San Francisco,No County Name County,86.0,Layoff Permanent,2015
9503,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Santa Monica,No County Name County,49.0,Closure Permanent,2015
9504,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Venice,No County Name County,11.0,Closure Permanent,2015
9505,06/29/2015,08/28/2015,06/30/2015,"Safeway, Inc.",Pleasanton,No County Name County,18.0,Layoff Unknown at this time,2015


In [20]:
no_county['Year'].value_counts()

2015    759
2016    704
2017    420
2014    231
2018      3
Name: Year, dtype: int64